### Машинное обучение в бизнесе
### КУРСОВОЙ ПРОЕКТ 

**Описание датасета**:    

Датасет взят из kaggle - https://www.kaggle.com/blackmoon/russian-language-toxic-comments  
Даны токсичные комментарии на русском языке - хорошие "нетоксичные" и токсичные.  
Датасет содержит всего 2 столбца - комментарий и метка токсичности.  
  
  
**Задача** - определить токсичность комментария  
  
Базовый класс - 0 (хороший, "нетоксичный" комментарий)  
Целевой класс - 1 (плохой, токсичный комметарий)

In [60]:
#cd ..

In [61]:
#!pip install flask-ngrok
#!pip install flask

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#для обработки текста
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from razdel import tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')
import pymorphy2  
from sklearn.feature_extraction.text import TfidfVectorizer

#для построения pipeline
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

#для построения модели
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

#для интерпретации результатов модели
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.metrics import f1_score

import urllib

import pickle, dill, json
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aleksandrinavatkina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aleksandrinavatkina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
df = pd.read_csv('labeled.csv')
df.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [64]:
df.dtypes

comment     object
toxic      float64
dtype: object

#### Изучим целевую переменную

In [65]:
#скорректируем тип целевой переменной
df['toxic'] = df['toxic'].astype('int8')

In [66]:
df['toxic'].value_counts(normalize=True)

0    0.66514
1    0.33486
Name: toxic, dtype: float64

Выраженный дисбаланс классов, хорошие комментарии явно преобладают над токсичными

#### Разобьем датасет на train и test

In [67]:
X_train, X_test, y_train, y_test = train_test_split(df['comment'], df['toxic'], test_size=0.30, random_state=1)

In [68]:
X_train.to_csv('X_train.csv')
X_test.to_csv('X_test.csv')

In [69]:
y_train.value_counts(normalize=True)

0    0.663362
1    0.336638
Name: toxic, dtype: float64

In [70]:
y_test.value_counts(normalize=True)

0    0.669288
1    0.330712
Name: toxic, dtype: float64

Баланс классов в тренировочной и тестовой выборке примерно совпадает.

#### Feature Engineering  

Датасет крайне небольшой - всего 2 колонки, поэтому основной задачей будет обработка текста. 

In [71]:
#Попробуем произвести обработку на одном примере
comment_example = X_train.iloc[1]
comment_example

'Игромания не переходит к Канобу, никоим образом. Гаджи - новый сотрудник, руководитель, но не владелец и не главред. И да, он создал Канобу и был его операционным директором, но не является им в настоящий момент. Нет никакой речи о пересечении между этими двумя площадками.\n'

In [72]:
#токены
tokens = word_tokenize(comment_example, language="russian")

#токены без пунктуации
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation]

#загрузим стоп слова из русского языка
russian_stop_words = stopwords.words("russian")

# выведем токены без стоп слов и без знаков препинания
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]

#Проведем стемматизацию - "отрежем" окончания и суффиксы, оставим только корень слова 
snowball = SnowballStemmer(language="russian")
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]

In [73]:
# выведем результат на примере одного комментария, чтобы понять, корректно ли произведена обработка

print(f"Исходный текст: {comment_example}\nТокены: {tokens} \n\nТокены без пунктуации: {tokens_without_punctuation} \
Токены без пунктуации и стоп слов: {tokens_without_stop_words_and_punctuation}\n\n\
Токены после стемминга: {stemmed_tokens}")

Исходный текст: Игромания не переходит к Канобу, никоим образом. Гаджи - новый сотрудник, руководитель, но не владелец и не главред. И да, он создал Канобу и был его операционным директором, но не является им в настоящий момент. Нет никакой речи о пересечении между этими двумя площадками.

Токены: ['Игромания', 'не', 'переходит', 'к', 'Канобу', ',', 'никоим', 'образом', '.', 'Гаджи', '-', 'новый', 'сотрудник', ',', 'руководитель', ',', 'но', 'не', 'владелец', 'и', 'не', 'главред', '.', 'И', 'да', ',', 'он', 'создал', 'Канобу', 'и', 'был', 'его', 'операционным', 'директором', ',', 'но', 'не', 'является', 'им', 'в', 'настоящий', 'момент', '.', 'Нет', 'никакой', 'речи', 'о', 'пересечении', 'между', 'этими', 'двумя', 'площадками', '.'] 

Токены без пунктуации: ['Игромания', 'не', 'переходит', 'к', 'Канобу', 'никоим', 'образом', 'Гаджи', 'новый', 'сотрудник', 'руководитель', 'но', 'не', 'владелец', 'и', 'не', 'главред', 'И', 'да', 'он', 'создал', 'Канобу', 'и', 'был', 'его', 'операционным',

In [74]:
# создадим функцию для обработки текста на основе протестированной обработки выше:

#для стемминга
snowball = SnowballStemmer(language="russian")

#загрузим русские стоп слова
russian_stop_words = stopwords.words("russian")

def tokenize_comment(comment: str, remove_stop_words: bool = True):
    tokens = word_tokenize(comment, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

#[SnowballStemmer(language="russian").stem(i) for i in word_tokenize(x, language="russian") \
#                if i not in string.punctuation and if i not in stopwords.words("russian")]


In [75]:
# Проверим как будет работать на нашем выбранном примере
tokenize_comment(comment_example)

['игроман',
 'переход',
 'каноб',
 'нико',
 'образ',
 'гадж',
 'нов',
 'сотрудник',
 'руководител',
 'владелец',
 'главред',
 'и',
 'созда',
 'каноб',
 'операцион',
 'директор',
 'явля',
 'настоя',
 'момент',
 'нет',
 'никак',
 'реч',
 'пересечен',
 'эт',
 'двум',
 'площадк']

Отлично, функция работает корректно

In [76]:
#Соберем эту фунцию в генератор
[SnowballStemmer(language="russian").stem(i) for i in word_tokenize('Привет как дела', language="russian") if (i not in string.punctuation and i not in stopwords.words("russian"))]

['привет', 'дел']

In [77]:
#создадим TFIDF Vetorizer  с использованием написанной выше функцией
vectorizer = TfidfVectorizer(tokenizer=lambda x: [SnowballStemmer(language="russian").stem(i) for i in word_tokenize(x, language="russian") if (i not in string.punctuation and i not in stopwords.words("russian"))])

#### Соберем PIPELINE и обучим модель

In [78]:
#СОБЕРЕМ PIPELINE

pipeline = Pipeline([
    ('vectorizer',TfidfVectorizer(tokenizer=lambda x: [SnowballStemmer(language="russian").stem(i) for i in word_tokenize(x, language="russian") if (i not in string.punctuation and i not in stopwords.words("russian"))])),
    ('model', LogisticRegression(random_state=0))
])

In [79]:
# Обучим pipeline

pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x7fc2303b3430>)),
                ('model', LogisticRegression(random_state=0))])

In [196]:
# Сделаем предсказание

y_predict = pipeline.predict_proba(X_test)[:,1]

In [197]:
precision, recall, thresholds = precision_recall_curve(y_test, y_predict)

fscore = (2 * precision * recall) / (precision + recall)
# выделим индекс с наибольшим значением fscore
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.359030, F-Score=0.817, Precision=0.775, Recall=0.864


Отлично, получили вполне хороший результат    
Помимо текущей модели, были протестированы "деревянные" ансамблевые модели - GradientBoosting и RandomForest, но даже с подбором гиперпараметров модели дали результат слабее, нежели Логистическая регрессия.

**Потестируем случайные придуманные комментарии**

In [82]:
pipeline.predict(['Привет как дела'])

array([0], dtype=int8)

In [184]:
pipeline.predict(['Я пошел в магазин'])

array([0], dtype=int8)

In [187]:
roc_auc_score(y_test, y_predict)

0.9344556618226281

#### Сохраним модель

In [86]:
with open("logreg_pipeline_.dill", "wb") as f:
    dill.dump(pipeline, f)

**Сохраним версионирование** 

In [87]:
versioning = {'date': '15.10.2021',
              'model': 'logreg_pipeline_.dill',
              'version': '1.0.0'
             }

In [88]:
with open("versioning.dill", "wb") as f:
    dill.dump(versioning, f)

#### Создадим образ и контейнер Docker

In [47]:
#pwd

список образов - docker images -a    
удаление образа - docker rmi Image Image  
список контейнеров - docker ps -a  
удаление контейнера - docker rm ID_or_Name ID_or_Name

In [48]:
#cd course_project

In [49]:
# Создаем образ
#!docker build -t classification_comments .

**Собираем контейнер**

In [50]:
#! docker run -d -p 8183:8183 -v /Users/aleksandrinavatkina/Desktop/GeekBrains/'Машинное обучение в бизнесе'/les9_cp/course_project/app/models:/app/app/models classification_comments

Сохраним id контейнера - 0780eb3bc34879dbf2b16e95eaa6ef825b6c58be055de3ee432c812c09904a9f

Готово! Собрали наш контейнер

**Проверяем работу и делаем предсказания**

In [89]:
X_test = pd.read_csv('/Users/aleksandrinavatkina/Desktop/GeekBrains/Машинное обучение в бизнесе/les9_cp/X_test.csv', index_col = 0, header=0, squeeze=True)

In [90]:
# Проверим как выглядит файл
X_test.head()

11824    А в районах победнее под ногами можно найти на...
5594                Весь мир Прям смешно стало, свинявый\n
135                           пруф или пи..бол шарлатан)\n
12046    Меня после жизни в СПб уже не пугают такие цен...
13038    Я тоже начал собирать пивные крышки, но только...
Name: comment, dtype: object

In [91]:
type(X_test)

pandas.core.series.Series

In [112]:
def get_prediction(comment):
    body = {'comment': [comment]} 
    myurl = "http://172.20.10.3:8183/classify" #наш url
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    #print (jsondataasbytes)
    response = urllib.request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [201]:
get_prediction("Жопа")

1

In [202]:
get_prediction('Я пошел в магазин. Спишемся позже')

0

In [204]:
predictions = X_test.iloc[:50].apply(lambda x: get_prediction(x))
predictions

11824    0
5594     0
135      0
12046    0
13038    0
13076    0
4468     0
7565     1
12203    1
6235     1
11526    0
4641     0
153      0
9531     0
13638    1
5831     1
1160     1
6922     0
9729     0
9314     0
3669     0
10073    0
5722     0
3773     0
9420     0
9288     0
12900    0
7137     0
4930     0
8854     0
12033    0
11498    0
8900     0
7682     1
8155     1
11402    0
9309     0
3144     0
8076     0
4913     0
6596     0
1217     0
8792     0
8395     0
10817    0
5222     0
6126     0
1949     0
8096     0
13714    0
Name: comment, dtype: int64